<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



Por: Sevann Radhak Triztan

# OBJETIVO  
El objetivo del código es descargar, procesar y analizar dos conjuntos de datos de noticias: uno de noticias falsas y el otro de noticias verdaderas, utilizando modelos de Word2Vec para crear representaciones vectoriales de las palabras en estos textos. El propósito principal es comparar las similitudes entre palabras en los contextos de noticias falsas y verdaderas, analizar sus posibles causas y visualizar estas diferencias.

In [ ]:
import os
import pandas as pd
import platform
import multiprocessing
from zipfile import ZipFile
from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import numpy as np
import plotly.express as px
from sklearn.manifold import TSNE

Constantes.

In [ ]:
BASE_URL = 'https://raw.githubusercontent.com/sevann-radhak/procesamiento_lenguaje_natural/main/clase_2/ejercicios/raw'
DESTINATION_FOLDER = './fake_news_dataset'
EPOCHS = 20
VECTOR_SIZE = 300
MIN_COUNT = 5
WINDOW = 2
NEGATIVE = 20
SG = 1
ALPHA = 0.01
MIN_ALPHA = 0.0001
MAX_WORDS = 200

Callback.

In [ ]:
class LossCallback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.loss_previous_step = 0
        self.stop_training = False

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print(f'Loss after epoch {self.epoch}: {loss - self.loss_previous_step}')
        if loss <= 0:
            print("Stopping training because loss reached zero.")
            self.stop_training = True
        self.epoch += 1
        self.loss_previous_step = loss

Funciones de procesos.

In [ ]:
def download_file(file_name, base_url, destination_folder):
    destination_path = os.path.join(destination_folder, file_name)
    if not os.path.exists(destination_path):
        full_url = os.path.join(base_url, file_name)
        os.system(f'curl {full_url} -o {file_name}' if platform.system() == 'Windows' else f'wget {full_url} -O {file_name}')
        os.makedirs(destination_folder, exist_ok=True)
        os.rename(file_name, destination_path)
    else:
        print(f"The file {file_name} is already downloaded")

def load_data(destination_folder):
    df_fake = pd.read_csv(os.path.join(destination_folder, 'Fake.csv'))
    df_true = pd.read_csv(os.path.join(destination_folder, 'True.csv'))

    for df in [df_fake, df_true]:
        df['content'] = df.apply(lambda row: row['title'] + ' ' + row['text'] if not row['title'].endswith('.') else row['title'] + row['text'], axis=1)
        df.drop(columns=['date', 'subject', 'title', 'text'], inplace=True)

    return df_fake, df_true

def tokenize_content(df):
    return [text_to_word_sequence(content) for content in df['content']]

def build_and_train_word2vec(tokens, vector_size, window, min_count, sg, alpha, min_alpha, epochs):
    model = Word2Vec(
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,
        alpha=alpha,
        min_alpha=min_alpha,
        workers=multiprocessing.cpu_count(),
        negative=NEGATIVE
    )
    model.build_vocab(tokens)
    model.train(tokens, total_examples=model.corpus_count, epochs=epochs, compute_loss=True, callbacks=[LossCallback()])
    return model

def compare_most_similar(word, model_fake, model_true, topn=10):
    fake_similar = model_fake.wv.most_similar(positive=[word], topn=topn)
    true_similar = model_true.wv.most_similar(positive=[word], topn=topn)

    print(f"Comparing most similar words for: '{word}'\n")
    print(f"{'Fake News':<20}{'True News':<20}")
    print(f"{'-'*40}")
    for fake_word, true_word in zip(fake_similar, true_similar):
        print(f"{fake_word[0]:<20}{true_word[0]:<20}")

def reduce_dimensions(model, num_dimensions=2):
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

def plot_word_vectors(vectors, labels, title, max_words=MAX_WORDS):
    fig = px.scatter(x=vectors[:max_words, 0], y=vectors[:max_words, 1], text=labels[:max_words], title=title)
    fig.show(renderer="colab")

Descarga de datos.

In [ ]:
download_file('Fake.csv', BASE_URL, DESTINATION_FOLDER)
download_file('True.csv', BASE_URL, DESTINATION_FOLDER)

The file Fake.csv is already downloaded
The file True.csv is already downloaded


Carga de datos a Dataframe.

In [ ]:
df_fake, df_true = load_data(DESTINATION_FOLDER)

Tokenización.

In [ ]:
tokens_fake = tokenize_content(df_fake)
tokens_true = tokenize_content(df_true)

Construcción y entrenamiento de modelo para FAKE.

In [ ]:
print("Training Word2Vec fake news model...")
w2v_fake_model = Word2Vec(
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    sg=SG,
    alpha=ALPHA,
    min_alpha=MIN_ALPHA,
    workers=multiprocessing.cpu_count(),
    negative=NEGATIVE
)

w2v_fake_model.build_vocab(tokens_fake)
callback_fake = LossCallback()
w2v_fake_model.train(
    tokens_fake,
    total_examples=w2v_fake_model.corpus_count,
    epochs=EPOCHS,
    compute_loss=True,
    callbacks=[callback_fake]
)

if callback_fake.stop_training:
    print("Training stopped because loss reached zero.")

Training Word2Vec fake news model...
Loss after epoch 0: 34471416.0
Loss after epoch 1: 32055512.0
Loss after epoch 2: 7862560.0
Loss after epoch 3: 7431888.0
Loss after epoch 4: 7386000.0
Loss after epoch 5: 7277424.0
Loss after epoch 6: 7225496.0
Loss after epoch 7: 7165856.0
Loss after epoch 8: 7040184.0
Loss after epoch 9: 7021304.0
Loss after epoch 10: 6870960.0
Loss after epoch 11: 2409128.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0


Construcción y entrenamiento de modelo para TRUE.

In [ ]:
print("Training Word2Vec true news model...")
w2v_true_model = Word2Vec(
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    sg=SG,
    alpha=ALPHA,
    min_alpha=MIN_ALPHA,
    workers=multiprocessing.cpu_count(),
    negative=NEGATIVE
)

w2v_true_model.build_vocab(tokens_true)
callback_true = LossCallback()
w2v_true_model.train(
    tokens_true,
    total_examples=w2v_true_model.corpus_count,
    epochs=EPOCHS,
    compute_loss=True,
    callbacks=[callback_true]
)

if callback_true.stop_training:
    print("Training stopped because loss reached zero.")

Training Word2Vec true news model...
Loss after epoch 0: 27652020.0
Loss after epoch 1: 24335176.0
Loss after epoch 2: 17265092.0
Loss after epoch 3: 6244048.0
Loss after epoch 4: 6135704.0
Loss after epoch 5: 6103024.0
Loss after epoch 6: 6008400.0
Loss after epoch 7: 5945560.0
Loss after epoch 8: 5888288.0
Loss after epoch 9: 5851176.0
Loss after epoch 10: 5764088.0
Loss after epoch 11: 5746688.0
Loss after epoch 12: 5694272.0
Loss after epoch 13: 5584192.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0


### Comparación de palabras más similares.

In [ ]:
compare_most_similar("media", w2v_fake_model, w2v_true_model)

Comparing most similar words for: 'media'

Fake News           True News           
----------------------------------------
mainstream          outlets             
outlets             newspapers          
blogs               networking          
dutiful             “recent             
medias              “chinese            
media”              mizan               
hypocritically      brandwatch          
pundits             zbc                 
msm                 outlet              
technocrats         bots                


In [ ]:
compare_most_similar("sex", w2v_fake_model, w2v_true_model)

Comparing most similar words for: 'sex'

Fake News           True News           
----------------------------------------
heterosexual        marriage            
anal                heterosexual        
interracial         homosexual          
oral                adultery            
marital             reassignment        
molesters           unprotected         
forcible            abuser              
molestation         couples             
homosexual          sexual              
sexual              prostitute          


In [ ]:
compare_most_similar("donald", w2v_fake_model, w2v_true_model)

Comparing most similar words for: 'donald'

Fake News           True News           
----------------------------------------
trump               trump               
2016donald          presumptive         
“president          vietnam's           
2017donald          “unfit”             
impeaching           trump              
bloviating          values'             
cthulhu             anoint              
you’d               ‘mr                 
majorly             ivanka              
hesitance           disavowal           


In [ ]:
compare_most_similar("russia", w2v_fake_model, w2v_true_model)

Comparing most similar words for: 'russia'

Fake News           True News           
----------------------------------------
russian             moscow              
russians            russian             
kremlin             russia’s            
putin               kremlin             
collusion           russians            
specious            'extremely          
ukraine             ukraine             
mending             russia's            
meddling            moscow’s            
moscow              putin               


In [ ]:
compare_most_similar("macri", w2v_fake_model, w2v_true_model)

Comparing most similar words for: 'macri'

Fake News           True News           
----------------------------------------
mauricio            mauricio            
duda                bachelet            
argentine           fernandez           
martelly            cambiemos           
jinping             cristina            
jae                 sirisena            
pussygrabber        argentina's         
yugoslav            duda                
nicol               moise               
xi                  muhammadu           


In [ ]:
compare_most_similar("politics", w2v_fake_model, w2v_true_model, topn=20)

Comparing most similar words for: 'politics'

Fake News           True News           
----------------------------------------
shushwalshe         demographics        
injecting           panicking           
rancor              tinge               
populism            contemporary        
fumbles             magnanimous         
zeitgeist           greatness           
compost             cabal               
jingoism            quintessential      
endgame             psyche              
sloppiness          polarized           
lifetimes           politician          
passivity           tricks              
pioneers            hardball            
sandbox             jibe                
disintegrated       huntsville          
nuances             tenor               
dichotomy           squabbles           
nativism            fluent              
oligarchy           technocrats         
projecting          benevolent          


In [ ]:
compare_most_similar("government", w2v_fake_model, w2v_true_model, topn=20)

Comparing most similar words for: 'government'

Fake News           True News           
----------------------------------------
governmental        govt                
councils            “federal            
bop                 government’s        
gubmint             ministry            
governments         “financial          
benefactors         airforce            
militaries          imperialism         
govt                vakifbank           
formulating         cypriots            
militarize          division’s          
nonviolently        ashraf              
discretionary       “u                  
largesse            'go                 
depletion           qaeda’s             
afoul               “waive              
unitary             attained            
unelected           beleaguered         
ceding              symbolized          
onerous             nia                 
quintessential      asayish             


In [ ]:
compare_most_similar("technology", w2v_fake_model, w2v_true_model, topn=20)

Comparing most similar words for: 'technology'

Fake News           True News           
----------------------------------------
technologies        tech                
advancements        robotics            
deterrence          wireless            
vocational          advancements        
hvac                vdma                
duplicative         telecommunications  
imaging             “corporate          
face2face           telecommunication   
simulations         technologies        
nuances             biotechnology       
expertise           aerospace           
software            programmers         
upstream            startup             
sonar               semiconductor       
compartmented       specialists         
macquarie           industrials         
breakthroughs       forestry            
ingenuity           automotive          
autocomplete        unlisted            
configuration       entrepreneurship    


 Reducción de  dimensiones and y gráficas.

In [ ]:
vecs_fake, labels_fake = reduce_dimensions(w2v_fake_model)
plot_word_vectors(vecs_fake, labels_fake, "Fake News Word Vectors")

vecs_true, labels_true = reduce_dimensions(w2v_true_model)
plot_word_vectors(vecs_true, labels_true, "True News Word Vectors")

In [ ]:
fake_vectors = np.asarray(w2v_fake_model.wv.vectors)
fake_labels = list(w2v_fake_model.wv.index_to_key)

np.savetxt("fake_vectors.tsv", fake_vectors, delimiter="\t")

with open("fake_labels.tsv", "w") as fp:
    for item in fake_labels:
        fp.write("%s\n" % item)

In [ ]:
true_vectors = np.asarray(w2v_true_model.wv.vectors)
true_labels = list(w2v_true_model.wv.index_to_key)

np.savetxt("true_vectors.tsv", true_vectors, delimiter="\t")

with open("true_labels.tsv", "w") as fp:
    for item in true_labels:
        fp.write("%s\n" % item)

# **CONCLUSIONES**  
El objetivo del trabajo fue crear representaciones vectoriales (embeddings) de palabras utilizando modelos de Word2Vec entrenados con noticias falsas y verdaderas. Para esto, se descargaron y procesaron dos conjuntos de datos (Fake.csv y True.csv) que contienen noticias falsas y verdaderas respectivamente. Se construyeron y entrenaron dos modelos de Word2Vec, uno para cada conjunto de datos, utilizando Gensim.  

Se seleccionaron varios términos de interés como "media", "sex", "donald", "russia", "macri", "politics", "government" y "technology" para comparar sus palabras más similares en los espacios de embeddings de noticias falsas y verdaderas.  

Al comparar los términos de interés en los espacios de embeddings, se observaron varias diferencias significativas en los contextos de uso entre las noticias falsas y verdaderas:  

* **"media":** En las noticias falsas, las palabras similares incluyen "mainstream" y "blogs", sugiriendo un enfoque crítico hacia los medios tradicionales. En las noticias verdaderas, los términos como "outlets" y "newspapers" reflejan un contexto más neutral.
* **"sex":** Las noticias falsas asocian "sex" con términos como "anal", "oral" e "interracial", indicando una posible tendencia hacia temas sensacionalistas. En cambio, las noticias verdaderas asocian "sex" con "marriage", "homosexual" y "couples", reflejando un enfoque más equilibrado.
* **"russia":** La presencia de términos como "collusion", "specious" y "meddling", en las noticias falsas, indica una tendencia a destacar teorías de conspiración y temas sensacionalistas en noticias falsa. Estas palabras también muestran una polarización fuerte, con un énfasis en desacreditar y generar dudas. En cambio, en las noticias verdaderas se destacan términos como "moscow", "kremlin", "russia’s", y "moscow’s", que reflejan un enfoque más geopolítico y formal centrado en lugares y entidades estatales.
* **"macri":** En las noticias falsas, se asocia "macri" con términos despectivos y figuras controversiales, mientras que en las noticias verdaderas se relaciona con nombres de políticos y eventos.
* **"politics":** La presencia de términos como "rancor", "populism", y "jingoism", en las noticias falsas, indica una tendencia hacia el sensacionalismo y el desdén hacia ciertos aspectos de la política. Además, términos como "fumbles", "sloppiness", y "compost" reflejan un enfoque crítico y descalificativo, presentando el contexto de la palabra "politics" de manera negativa. En las noticias verdaderas, encontramos t
términos como "demographics", "contemporary", y "psyche", que reflejan un enfoque más descriptivo y analítico, presentando la palabra "politics" en un contexto de estudio y observación.
* **"government":** Las noticias falsas incluyen términos como "gubmint" y "govt", mientras que las verdaderas usan "government’s" y "ministry", reflejando un enfoque más formal y descriptivo.
* **"technology":** Las noticias falsas asocian "technology" con términos como "advancements" y "deterrence", mientras que las verdaderas incluyen términos como "robotics", "wireless", y "biotechnology" que reflejan un enfoque en áreas específicas y relevantes de la tecnología, mostrando una narrativa basada en hechos y desarrollos actuales.  



**Conclusiones Generales**

1. **Diferencias en Narrativas:** las noticias falsas tienden a utilizar un lenguaje polarizante, sensacionalista y despectivo, mientras que las noticias verdaderas se esfuerzan por mantener una representación equilibrada, objetiva y basada en hechos.
2. **Polarización:** la comparación de términos muestra una polarización clara en cómo se presentan ciertos temas en noticias falsas vs. verdaderas, especialmente en áreas de política, tecnología y medios de comunicación.
3. **Impacto en la Percepción:** las diferencias en las asociaciones de palabras entre noticias falsas y verdaderas pueden influir significativamente en el impacto de la percepción del mensaje transmitido.









